In [1]:
# encoder image test

###### MATF TEST #######

# Imports
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torchvision
import numpy as np
import torch.nn.functional as F
from utils import weights_init, conv2DBatchNormRelu, conv2DRelu, deconv2DBatchNormRelu, deconv2DRelu
import torchvision.models as models
import matplotlib.pyplot as plt
import torchvision.transforms as T
import os
from PIL import Image

In [22]:
class ImageDataset(Dataset):
    """
    Creates a PyTorch dataset from folder, returning two tensor images.
    Args: 
    main_dir : directory where images are stored.
    transform (optional) : torchvision transforms to be applied while making dataset
    """

    def __init__(self, main_dir, transform=None):
        self.main_dir = main_dir
        self.transform = transform
        self.all_imgs = os.listdir(main_dir)
        
        img_list=[]
        img_tensor_list=[]
        
        all_files = os.listdir(self.main_dir)
        
        for file in all_files:
            path = main_dir+file
            image = Image.open(path).convert("RGB")
            if self.transform is not None:
                tensor_image = self.transform(image)
            img_list.append(image)
            img_tensor_list.append(tensor_image)

            
        
        self.list_img = img_list
        self.list_ten_img = img_tensor_list

        

            
        
    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, index):
        out =  [
                self.list_img,
                self.list_ten_img
        ]

        return out

In [23]:
transforms = T.Compose([T.ToTensor()]) # Normalize the pixels and convert to tensor.
full_dataset = ImageDataset(
    "C:/Users/NGN/dev/Traffino/TRAFFINO/traffino/datasets/waterloo/img_test/769/", 
    transforms
    ) # Create folder dataset.

In [35]:
print(full_dataset[1][1])


[tensor([[[0.0000, 0.0039, 0.0039,  ..., 0.2667, 0.2392, 0.2510],
         [0.0000, 0.0039, 0.0039,  ..., 0.2549, 0.2275, 0.2353],
         [0.0000, 0.0000, 0.0039,  ..., 0.2314, 0.2078, 0.2235],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0039, 0.0118, 0.0039,  ..., 0.3255, 0.3137, 0.3255],
         [0.0039, 0.0118, 0.0039,  ..., 0.3294, 0.3059, 0.3137],
         [0.0039, 0.0078, 0.0039,  ..., 0.3176, 0.2941, 0.3176],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0078, 0.0000,  ..., 0.0941, 0.0627, 0.0667],
         [0.0000, 0.0078, 0.0000,  ..., 0.1137, 0.0745, 0.0745],
         [0.0000, 0.0039, 0.0000,  ..., 0.1373, 0.1020, 0

In [40]:
print(full_dataset[2][1])
print(type(full_dataset[2][1]))
print(len(full_dataset[2][1]))
print(len(full_dataset[2][1]))

[tensor([[[0.0000, 0.0039, 0.0039,  ..., 0.2667, 0.2392, 0.2510],
         [0.0000, 0.0039, 0.0039,  ..., 0.2549, 0.2275, 0.2353],
         [0.0000, 0.0000, 0.0039,  ..., 0.2314, 0.2078, 0.2235],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0039, 0.0118, 0.0039,  ..., 0.3255, 0.3137, 0.3255],
         [0.0039, 0.0118, 0.0039,  ..., 0.3294, 0.3059, 0.3137],
         [0.0039, 0.0078, 0.0039,  ..., 0.3176, 0.2941, 0.3176],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0078, 0.0000,  ..., 0.0941, 0.0627, 0.0667],
         [0.0000, 0.0078, 0.0000,  ..., 0.1137, 0.0745, 0.0745],
         [0.0000, 0.0039, 0.0000,  ..., 0.1373, 0.1020, 0

In [45]:
a=full_dataset[2][1][0]
print(a)
print(type(a))
print(a.shape)


tensor([[[0.0000, 0.0039, 0.0039,  ..., 0.2667, 0.2392, 0.2510],
         [0.0000, 0.0039, 0.0039,  ..., 0.2549, 0.2275, 0.2353],
         [0.0000, 0.0000, 0.0039,  ..., 0.2314, 0.2078, 0.2235],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0039, 0.0118, 0.0039,  ..., 0.3255, 0.3137, 0.3255],
         [0.0039, 0.0118, 0.0039,  ..., 0.3294, 0.3059, 0.3137],
         [0.0039, 0.0078, 0.0039,  ..., 0.3176, 0.2941, 0.3176],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0078, 0.0000,  ..., 0.0941, 0.0627, 0.0667],
         [0.0000, 0.0078, 0.0000,  ..., 0.1137, 0.0745, 0.0745],
         [0.0000, 0.0039, 0.0000,  ..., 0.1373, 0.1020, 0.

In [61]:
class SemanticImageEncoder(nn.Module):
    '''
    Tianyang:
    Simple Convolutional Encoder for semantic map images
    Input size: flexible
    Output size: 1/2 input
    '''

    def __init__(self, in_channels = 3, out_channels = 32):
        super(SemanticImageEncoder, self).__init__()

        self._encoder = nn.Sequential(
            conv2DBatchNormRelu(in_channels = in_channels, n_filters = 16, \
                k_size = 3,  stride = 1, padding = 1),
            conv2DBatchNormRelu(in_channels = 16, n_filters = 16, \
                k_size = 4,  stride = 1, padding = 2),
            nn.MaxPool2d((2, 2), stride=(2, 2)),

            conv2DBatchNormRelu(in_channels = 16, n_filters = out_channels, \
                k_size = 5,  stride = 1, padding = 2),
            )

    def forward(self, input):
        encoded = self._encoder(input.type(torch.FloatTensor))
        return encoded

In [63]:
image_channel=3
embed_dim_image=32
image_encoder = SemanticImageEncoder(in_channels = image_channels, out_channels = embed_dim_image)

In [69]:
a_s=a.unsqueeze(0)

print(a_s)
print(type(a_s))
print(a_s.shape)

tensor([[[[0.0000, 0.0039, 0.0039,  ..., 0.2667, 0.2392, 0.2510],
          [0.0000, 0.0039, 0.0039,  ..., 0.2549, 0.2275, 0.2353],
          [0.0000, 0.0000, 0.0039,  ..., 0.2314, 0.2078, 0.2235],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0039, 0.0118, 0.0039,  ..., 0.3255, 0.3137, 0.3255],
          [0.0039, 0.0118, 0.0039,  ..., 0.3294, 0.3059, 0.3137],
          [0.0039, 0.0078, 0.0039,  ..., 0.3176, 0.2941, 0.3176],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0078, 0.0000,  ..., 0.0941, 0.0627, 0.0667],
          [0.0000, 0.0078, 0.0000,  ..., 0.1137, 0.0745, 0.0745],
          [0.0000, 0.0039, 0.0000,  ..., 0

In [71]:
f_a=image_encoder(a_s)
print(f_a)

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.4375, 0.5683,  ..., 0.5473, 0.2667, 0.0000],
          [0.0000, 0.6425, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.8654, 1.4600,  ..., 1.0844, 0.9565, 1.4219],
          [0.0000, 1.1778, 0.5397,  ..., 0.7689, 0.8212, 1.5740],
          [0.0000, 0.4034, 0.0000,  ..., 0.0000, 0.0000, 0.0477]],

         [[0.0000, 0.5877, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0972, 0.3027, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.5271, 1.1554, 0.0000,  ..., 0.0000, 0.3959, 1.7221],
          ...,
          [0.0000, 0.0083, 0.0000,  ..., 0.0000, 0.0000, 0.2568],
          [0.0000, 0.0529, 0.0000,  ..., 0.0000, 0.4970, 0.7239],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2107]],

         [[0.0989, 0.6687, 0.8065,  ..., 0.1440, 0.0000, 0.0000],
          [0.0000, 0.9916, 0.6189,  ..., 0.0000, 0.0000, 0.0000],
          [0.4452, 0.7426, 0.6440,  ..., 0

In [7]:
def list2batch(seq):
    # assemble a list of elements to batch, batch_idx: 0th dimension
    stacked = torch.tensor(seq[0]).unsqueeze(0)
    i = 1
    l = len(seq)
    while i < l:
        stacked = torch.cat((stacked, torch.tensor(seq[i]).unsqueeze(0)), 0)
        i += 1
    return stacked

In [8]:
# input_list should be a list, whose each element comes from a scene:
# input_list
# num_scenes

In [36]:
def seq_collate(data):
    (
        list_img, 
        list_ten_img, 
    ) = zip(*data)

    _len = [len(seq) for seq in obs_seq_list]
    cum_start_idx = [0] + np.cumsum(_len).tolist()
    seq_start_end = [[start, end]
                     for start, end in zip(cum_start_idx, cum_start_idx[1:])]

    # Data format: batch, input_size, seq_len
    # LSTM input format: seq_len, batch, input_size
    img_ = torch.cat(list_img, dim=0)

    img_tensor = torch.cat(list_ten_img, dim=0)

    
    out = [
        img_, 

        img_tensor, 

    ]

    return tuple(out)

In [ ]:
input_image_list=[]
for i in range(num_scenes):
    input_image_list.append(input_list[i][2][0])
    
input_image_batch = list2batch(input_image_list)
embed_image_batch=image_encoder(input_image_batch.cuda())

In [ ]:
class MultiAgentSceneTest(nn.Module):
    def __init__(self, image_channels=3, embed_dim_image = 32):
        
        super(MultiAgentSceneTest, self).__init__()
        
        self._semantic_image_encoder = SemanticImageEncoder(in_channels = image_channels, out_channels = embed_dim_image)
        self._resnet = resnetShallow()
 
    def forward(self, config, num_scenes, 
            input_list, 
            resample=0, std=1, use_resnet=0):
        # input_list should be a list, whose each element comes from a scene:   


        input_image_list = []
        for i in range(num_scenes):
            input_image_list.append(input_list[i][2][0])
        
        input_image_batch = self.list2batch(input_image_list)
        
        if use_resnet == 0:
            embed_image_batch = self._semantic_image_encoder(input_image_batch.cuda())
        else:
            embed_image_batch = self._resnet(input_image_batch.cuda())
        
        